In [1]:
import h5py
filenameCamera = '/home/is-lab/Desktop/Untitled Folder/Project/camera/2016-04-21--14-48-08.h5'
fCamera = h5py.File(filenameCamera, 'r')

list (fCamera)

# List all groups
#print("Keys: %s" % f.keys())
#a_group_key = list(f.keys())[0]

# Get the data
#data = list(f[a_group_key])
#print (data)

['X']

In [2]:
fCamera['X'].shape

(30538, 3, 160, 320)

In [3]:
import h5py
filenameLog = '/home/is-lab/Desktop/Untitled Folder/Project/log/2016-04-21--14-48-08.h5'
fLog = h5py.File(filenameLog, 'r')

#list (fLog)


In [4]:

a = fLog['cam1_ptr']
#len([i for i, e in enumerate(a) if e != 0])
a.shape

(152650,)

In [5]:
a1 = list(a)
#print (a1)
#print(a[3000:4000])
#a.index("621")
#print(a[294916])

In [6]:
a1.index(621)

3110

In [7]:
fLog['speed'][3148]

-0.2787096614676553

In [8]:
prev = 0
k = 0
sumSpeed = 0
sumSteering = 0

l = 0
mSpeed = list()
mSteering = list()

for i in a:
    if i!=prev:
        mSpeed.append(sumSpeed/l)
        mSteering.append(sumSteering/l)
        l = 1
        sumSpeed = fLog['speed'][k];
        sumSteering = fLog['steering_angle'][k];
    else:
        sumSpeed = fLog['speed'][k];
        sumSteering = fLog['steering_angle'][k];
        l = l+1
    prev = i
    k = k+1
    
print(len(mSpeed))
print(len(mSteering))

30538
30538


In [9]:
#len(m)

In [10]:
image = fCamera['X'][4090,:,:,:]
print(image.shape)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

def rgb2gray(rgb):
    r, g, b = rgb[0,:,:], rgb[1,:,:], rgb[2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    #return np.dot(rgb[:3,:,:], [0.299, 0.587, 0.114])
    return gray
     
from scipy.misc import toimage
from scipy.misc import imshow
#imshow(image)

gray = rgb2gray(image)  
#imshow(gray)

(3, 160, 320)


In [11]:
x = fCamera['X'].value
ySpeed = np.asarray(mSpeed)
ySteering = np.asarray(mSteering)
fCamera.close()
fLog.close()

#x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)


/home/is-lab/.local/lib/python3.5/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [12]:
from sklearn.utils import shuffle
x, ySpeed, ySteering = shuffle(x, ySpeed, ySteering, random_state=0)

In [13]:
#c1 = np.c_[x.reshape(len(x), -1), ySpeed.reshape(len(ySpeed), -1)]
#x = c1[:, :x.size//len(x)].reshape(x.shape)
#ySpeed = c1[:, x.size//len(ySpeed):].reshape(ySpeed.shape)

#c2 = np.c_[x.reshape(len(x), -1), ySteering.reshape(len(ySteering), -1)]

In [14]:
#c2 = np.c_[x.reshape(len(x), -1), ySteering.reshape(len(ySteering), -1)]
#ySteering = c2[:, x.size//len(ySteering):].reshape(ySteering.shape)

print(x.shape, ySpeed.shape, ySteering.shape)

(30538, 3, 160, 320) (30538,) (30538,)


In [16]:
x_train = x[0:25000, :, :, :]
x_test = x[25000:30538, :, :, :]

ySpeed.reshape(30538, 1)
ySteering.reshape(30538, 1)

ySpeed_train = ySpeed[0:25000]
ySpeed_test = ySpeed[25000:30538]
ySteering_train = ySteering[0:25000]
ySteering_test = ySteering[25000:30538]

print (x_train.shape)
print(x_test.shape)
print(len(ySpeed_train))
print(len(ySpeed_test))

print(len(ySteering_train))
print(len(ySteering_test))

(25000, 3, 160, 320)
(5538, 3, 160, 320)
25000
5538
25000
5538


In [17]:
x_train = np.rollaxis(x_train, 1, 4)
x_test = np.rollaxis(x_test, 1, 4)
print(x_train.shape)
x_test.shape

(25000, 160, 320, 3)


(5538, 160, 320, 3)

In [18]:
#x_train = x_train  / 255.0
#x_test = x_test / 255.0
#memory error
#np.column_stack((a,b))

y_train = np.column_stack((ySpeed_train, ySteering_train))
y_test = np.column_stack((ySpeed_test, ySteering_test))

print(y_train.shape)

(25000, 2)


In [ ]:
import tensorflow as tf
def multitask_loss(y_true, y_pred):
    lossSpeed = tf.losses.mean_squared_error(y_true[:,0], y_pred[:,0])
    lossSteering = tf.losses.mean_squared_error(y_true[:,1], y_pred[:,1])
    
    jointLoss = lossSpeed + lossSteering 
    print (lossSpeed, lossSteering)
    return jointLoss
    

In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, Input
from keras.layers import Conv2D, MaxPooling2D, Activation 
from keras.layers.convolutional import Convolution2D
from keras.models import Model

batch_size = 64
num_classes = 5
epochs = 10

# input image dimensions
img_rows, img_cols = 160, 320
channels = 3

#model = Sequential()

main_input = Input(shape=(img_rows, img_cols, channels), dtype='float32', name='main_input')
lambda1 = Lambda(lambda x: x/127.5 - 1.,
            input_shape=(img_rows, img_cols, channels),
            output_shape=(img_rows, img_cols, channels))(main_input)
convShared1 = Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same")(lambda1)
eluShared1 = ELU()(convShared1)
convShared2 = Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same")(eluShared1)
eluShared2 = ELU()(convShared2)


SteeringConv = Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same")(eluShared2)
SteeringFlat = Flatten()(SteeringConv)
SteeringDrop1 = Dropout(.2)(SteeringFlat)
SteeringElu1 = ELU()(SteeringDrop1)
SteeringDense = Dense(512)(SteeringElu1)
SteeringDrop2 = Dropout(.5)(SteeringDense)
SteeringElu2 = ELU()(SteeringDrop2)
steering = Dense(1)(SteeringElu2)


SpeedPool1 = MaxPooling2D(pool_size=(2, 2))(eluShared2)
SpeedDrop1 = Dropout(0.25)(SpeedPool1)

SpeedConv1 = Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="same")(SpeedDrop1)
SpeedRelu1 = Activation('relu')(SpeedConv1)
SpeedConv2 = Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="same")(SpeedRelu1)
SpeedRelu2 = Activation('relu')(SpeedConv2)
SpeedPool2 = MaxPooling2D(pool_size=(2, 2))(SpeedRelu2)
SpeedDrop2 = Dropout(0.25)(SpeedPool2)

SpeedFlat = Flatten()(SpeedDrop2)
SpeedDense = Dense(512)(SpeedFlat)
SpeedRelu3 = Activation('relu')(SpeedDense)
SpeedDrop3 = Dropout(0.5)(SpeedRelu3)
speed = Dense(1, kernel_initializer='normal')(SpeedDrop3)


model = Model(inputs=main_input, outputs=[speed , steering])

model.compile(optimizer='adam', loss='mean_squared_error',
              loss_weights=[1., 1.])
#model.compile(loss=multitask_loss,
#              optimizer='adam')

model.fit(x_train, [y_train[:,0],y_train[:,1] ],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, [y_test[:,0],y_test[:,1] ]))

Using TensorFlow backend.
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (8, 8), padding="same", strides=(4, 4))`
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), padding="same", strides=(2, 2))`
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same", strides=(2, 2))`
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", strides=(1, 1))`
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", strides=(1, 1))`


Train on 25000 samples, validate on 5538 samples
Epoch 1/10
25000/25000 [==============================] - 175s 7ms/step - loss: 19051.5726 - dense_4_loss: 1.4004 - dense_2_loss: 19050.1722 - val_loss: 17489.1557 - val_dense_4_loss: 0.6118 - val_dense_2_loss: 17488.5439
Epoch 2/10
25000/25000 [==============================] - 174s 7ms/step - loss: 11667.4013 - dense_4_loss: 0.6116 - dense_2_loss: 11666.7897 - val_loss: 8938.2907 - val_dense_4_loss: 0.2718 - val_dense_2_loss: 8938.0189
Epoch 3/10
25000/25000 [==============================] - 175s 7ms/step - loss: 7083.1021 - dense_4_loss: 0.4224 - dense_2_loss: 7082.6796 - val_loss: 6094.6726 - val_dense_4_loss: 0.1786 - val_dense_2_loss: 6094.4940
Epoch 4/10
25000/25000 [==============================] - 175s 7ms/step - loss: 5465.5903 - dense_4_loss: 0.3466 - dense_2_loss: 5465.2437 - val_loss: 5189.9357 - val_dense_4_loss: 0.1216 - val_dense_2_loss: 5189.8141
Epoch 5/10
25000/25000 [==============================] - 175s 7ms/step -

In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model20epochsshuffledMultitaskcombiningBestModelsBigFilter.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model20epochsshuffledMultitaskcombiningBestModelsBigFilter.h5")
print("Saved model to disk")


In [ ]:
#model.evaluate(x_test, y_test)

In [20]:
y_predict = model.predict(x_test)

In [ ]:
y_predict[0]

In [21]:
from sklearn.metrics import r2_score
print('Speed', r2_score(y_test[:,0], y_predict[0]))
print('Steering', r2_score(y_test[:,1], y_predict[1]))

Speed 0.9781598262585368
Steering 0.8850607501135151


In [22]:
x = np.rollaxis(x, 1, 4)

y_predictWhole = model.predict(x)

In [23]:
print(r2_score(ySpeed, y_predictWhole[0]), r2_score(ySteering, y_predictWhole[1]))


0.9818276356576758 0.9124442801151784
